<a href="https://colab.research.google.com/github/dumoura/gofai/blob/main/Good_Old_Fashioned_Artificial_Intelligence_(GOFAI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GOFAI

The main purpose here is to demonstrate how traditional symbolic AI techniques- Good Old-Fashioned Artificial Intelligence (GOFAI) - can be used to solve logical reasoning problems.

These examples illustrate key concepts in propositional logic and constraint satisfaction, which are fundamental to understanding symbolic AI.



### Key Objectives and Concepts Demonstrated:
1. **Logical Representation of Knowledge**:
   - The codes show how different real-world problems can be translated into a formal representation using symbols and logical sentences. For instance, people, rooms, and weapons are represented as symbols in the *Clue* example, while colors and positions are represented in the logic puzzle.
   - This symbolic representation allows AI to reason about the relationships and constraints without relying on statistical methods or machine learning.

2. **Using Propositional Logic for Inference**:
   - Each code example constructs a knowledge base consisting of logical sentences (propositional formulas). These sentences express rules, constraints, or facts about the problem.
   - The use of `And`, `Or`, `Not`, `Implication`, and `Biconditional` classes allows us to represent complex logical relationships between propositions.

3. **Constraint Satisfaction Problems (CSPs)**:
   - The codes present problems that can be viewed as CSPs, where the goal is to find values for variables that satisfy a set of constraints.
   - For example, in the *Clue* game scenario, the constraints ensure that exactly one person, one room, and one weapon are involved in the crime. Similarly, the logic puzzle with colors ensures that each color has a unique position.

4. **Model Checking for Logical Entailment**:
   - The `model_check` function illustrates the process of evaluating whether a knowledge base entails a given proposition. This involves checking all possible models (combinations of truth values for symbols) to see if the knowledge logically implies a specific outcome.
   - This approach exemplifies how symbolic AI performs reasoning based on formal logic rather than learned patterns, as in machine learning.

5. **Automated Deduction and Problem Solving**:
   - The examples simulate automated deduction by deriving conclusions from a set of premises. This is a core concept in symbolic AI, where the system uses logical rules to infer new knowledge.
   - For instance, the code can deduce which character, room, or weapon is involved in the *Clue* game by systematically eliminating impossible options.

### Teaching GOFAI Through These Examples:
As a professor teaching GOFAI, you would use these examples to help students understand how early AI approaches attempted to mimic human reasoning using symbolic representations and logical rules. The focus is on:
- **Encoding knowledge explicitly** as logical formulas, which can then be manipulated algorithmically.
- **Understanding the limitations and challenges** of symbolic reasoning, such as the need for exhaustive search (checking all models), scalability issues, and handling uncertainty or incomplete information.
- **Highlighting the contrast between GOFAI and modern AI approaches**, like machine learning, where reasoning is achieved through data-driven pattern recognition rather than explicit logical inference.

These examples serve as a practical introduction to symbolic AI, demonstrating how problems can be solved using logical reasoning techniques and setting the stage for more advanced topics in AI, such as knowledge representation, theorem proving, and non-monotonic reasoning.

In [ ]:
from IPython.display import display, HTML, Markdown
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import itertools

class Sentence:
    def evaluate(self, model):
        """Evaluates the logical sentence."""
        raise Exception("nothing to evaluate")

    def formula(self):
        """Returns string formula representing logical sentence."""
        return ""

    def symbols(self):
        """Returns a set of all symbols in the logical sentence."""
        return set()

    @classmethod
    def validate(cls, sentence):
        if not isinstance(sentence, Sentence):
            raise TypeError("must be a logical sentence")

    @classmethod
    def parenthesize(cls, s):
        """Parenthesizes an expression if not already parenthesized."""
        def balanced(s):
            """Checks if a string has balanced parentheses."""
            count = 0
            for c in s:
                if c == "(":
                    count += 1
                elif c == ")":
                    if count <= 0:
                        return False
                    count -= 1
            return count == 0

        if not len(s) or s.isalpha() or (
            s[0] == "(" and s[-1] == ")" and balanced(s[1:-1])
        ):
            return s
        else:
            return f"({s})"

class Symbol(Sentence):
    def __init__(self, name):
        self.name = name

    def __eq__(self, other):
        return isinstance(other, Symbol) and self.name == other.name

    def __hash__(self):
        return hash(("symbol", self.name))

    def __repr__(self):
        return self.name

    def evaluate(self, model):
        try:
            return bool(model[self.name])
        except KeyError:
            raise Exception(f"variable {self.name} not in model")

    def formula(self):
        return self.name

    def symbols(self):
        return {self.name}

class Not(Sentence):
    def __init__(self, operand):
        Sentence.validate(operand)
        self.operand = operand

    def __eq__(self, other):
        return isinstance(other, Not) and self.operand == other.operand

    def __hash__(self):
        return hash(("not", hash(self.operand)))

    def __repr__(self):
        return f"Not({self.operand})"

    def evaluate(self, model):
        return not self.operand.evaluate(model)

    def formula(self):
        return "¬" + Sentence.parenthesize(self.operand.formula())

    def symbols(self):
        return self.operand.symbols()

class And(Sentence):
    def __init__(self, *conjuncts):
        for conjunct in conjuncts:
            Sentence.validate(conjunct)
        self.conjuncts = list(conjuncts)

    def __eq__(self, other):
        return isinstance(other, And) and self.conjuncts == other.conjuncts

    def __hash__(self):
        return hash(("and", tuple(hash(conjunct) for conjunct in self.conjuncts)))

    def __repr__(self):
        conjunctions = ", ".join([str(conjunct) for conjunct in self.conjuncts])
        return f"And({conjunctions})"

    def add(self, conjunct):
        Sentence.validate(conjunct)
        self.conjuncts.append(conjunct)

    def evaluate(self, model):
        return all(conjunct.evaluate(model) for conjunct in self.conjuncts)

    def formula(self):
        if len(self.conjuncts) == 1:
            return self.conjuncts[0].formula()
        return " ∧ ".join([Sentence.parenthesize(conjunct.formula()) for conjunct in self.conjuncts])

    def symbols(self):
        return set.union(*[conjunct.symbols() for conjunct in self.conjuncts])

class Or(Sentence):
    def __init__(self, *disjuncts):
        for disjunct in disjuncts:
            Sentence.validate(disjunct)
        self.disjuncts = list(disjuncts)

    def __eq__(self, other):
        return isinstance(other, Or) and self.disjuncts == other.disjuncts

    def __hash__(self):
        return hash(("or", tuple(hash(disjunct) for disjunct in self.disjuncts)))

    def __repr__(self):
        disjuncts = ", ".join([str(disjunct) for disjunct in self.disjuncts])
        return f"Or({disjuncts})"

    def evaluate(self, model):
        return any(disjunct.evaluate(model) for disjunct in self.disjuncts)

    def formula(self):
        if len(self.disjuncts) == 1:
            return self.disjuncts[0].formula()
        return " ∨ ".join([Sentence.parenthesize(disjunct.formula()) for disjunct in self.disjuncts])

    def symbols(self):
        return set.union(*[disjunct.symbols() for disjunct in self.disjuncts])

class Implication(Sentence):
    def __init__(self, antecedent, consequent):
        Sentence.validate(antecedent)
        Sentence.validate(consequent)
        self.antecedent = antecedent
        self.consequent = consequent

    def __eq__(self, other):
        return (isinstance(other, Implication) and
                self.antecedent == other.antecedent and
                self.consequent == other.consequent)

    def __hash__(self):
        return hash(("implies", hash(self.antecedent), hash(self.consequent)))

    def __repr__(self):
        return f"Implication({self.antecedent}, {self.consequent})"

    def evaluate(self, model):
        return (not self.antecedent.evaluate(model) or self.consequent.evaluate(model))

    def formula(self):
        antecedent = Sentence.parenthesize(self.antecedent.formula())
        consequent = Sentence.parenthesize(self.consequent.formula())
        return f"{antecedent} => {consequent}"

    def symbols(self):
        return set.union(self.antecedent.symbols(), self.consequent.symbols())

class Biconditional(Sentence):
    def __init__(self, left, right):
        Sentence.validate(left)
        Sentence.validate(right)
        self.left = left
        self.right = right

    def __eq__(self, other):
        return (isinstance(other, Biconditional) and
                self.left == other.left and
                self.right == other.right)

    def __hash__(self):
        return hash(("biconditional", hash(self.left), hash(self.right)))

    def __repr__(self):
        return f"Biconditional({self.left}, {self.right})"

    def evaluate(self, model):
        return ((self.left.evaluate(model) and self.right.evaluate(model)) or
                (not self.left.evaluate(model) and not self.right.evaluate(model)))

    def formula(self):
        left = Sentence.parenthesize(self.left.formula())
        right = Sentence.parenthesize(self.right.formula())
        return f"{left} <=> {right}"

    def symbols(self):
        return set.union(self.left.symbols(), self.right.symbols())

def model_check(knowledge, query):
    """Checks if knowledge base entails query."""
    def check_all(knowledge, query, symbols, model):
        """Checks if knowledge base entails query, given a particular model."""
        # If model has an assignment for each symbol
        if not symbols:
            # If knowledge base is true in model, then query must also be true
            if knowledge.evaluate(model):
                return query.evaluate(model)
            return True
        else:
            # Choose one of the remaining unused symbols
            remaining = symbols.copy()
            p = remaining.pop()
            # Create a model where the symbol is true
            model_true = model.copy()
            model_true[p] = True
            # Create a model where the symbol is false
            model_false = model.copy()
            model_false[p] = False
            # Ensure entailment holds in both models
            return (check_all(knowledge, query, remaining, model_true) and
                    check_all(knowledge, query, remaining, model_false))

    # Get all symbols in both knowledge and query
    symbols = set.union(knowledge.symbols(), query.symbols())
    # Check that knowledge entails query
    return check_all(knowledge, query, symbols, dict())



Entailed symbols:
red0
blue1
yellow2
green3


In [ ]:
# Main block for Google Colab execution
if __name__ == "__main__":
    colors = ["red",  "green", "yellow", "blue"]
    symbols = [Symbol(f"{color}{i}") for i in range(4) for color in colors]
    knowledge = And()

    for color in colors:
        knowledge.add(Or(
            Symbol(f"{color}0"),
            Symbol(f"{color}1"),
            Symbol(f"{color}2"),
            Symbol(f"{color}3")
        ))

    # Only one position per color.
    for color in colors:
        for i in range(4):
            for j in range(4):
                if i != j:
                    knowledge.add(Implication(
                        Symbol(f"{color}{i}"), Not(Symbol(f"{color}{j}"))
                    ))

    # Only one color per position.
    for i in range(4):
        for c1 in colors:
            for c2 in colors:
                if c1 != c2:
                    knowledge.add(Implication(
                        Symbol(f"{c1}{i}"), Not(Symbol(f"{c2}{i}"))
                    ))

    # Adding some additional constraints to the knowledge base
    knowledge.add(Or(
        And(Symbol("red0"), Symbol("blue1"), Not(Symbol("green2")), Not(Symbol("yellow3"))),
        And(Symbol("red0"), Symbol("green2"), Not(Symbol("blue1")), Not(Symbol("yellow3"))),
        And(Symbol("red0"), Symbol("yellow3"), Not(Symbol("blue1")), Not(Symbol("green2"))),
        And(Symbol("blue1"), Symbol("green2"), Not(Symbol("red0")), Not(Symbol("yellow3"))),
        And(Symbol("blue1"), Symbol("yellow3"), Not(Symbol("red0")), Not(Symbol("green2"))),
        And(Symbol("green2"), Symbol("yellow3"), Not(Symbol("red0")), Not(Symbol("blue1")))
    ))

    # Further constraints for the knowledge base
    knowledge.add(And(
        Not(Symbol("blue0")),
        Not(Symbol("red1")),
        Not(Symbol("green2")),
        Not(Symbol("yellow3"))
    ))

    # Checking which symbols are entailed by the knowledge base
    print("Entailed symbols:")
    for symbol in symbols:
        if model_check(knowledge, symbol):
            print(symbol)


Entailed symbols:
red0
blue1
yellow2
green3


# Harry Potter People and Houses

In [ ]:
rain = Symbol("rain")
hagrid = Symbol("hagrid")
dumbledore = Symbol("dumbledore")

knowledge = And(
    Implication(Not(rain), hagrid),
    Or(hagrid, dumbledore),
    Not(And(hagrid, dumbledore)),
    dumbledore
)

print(model_check(knowledge, rain))


True


This code is a logic-based problem similar to assigning students to houses in a school (inspired by the Hogwarts houses from the Harry Potter universe). It uses propositional logic to set up and solve constraints regarding which people (professors) belong to which house. Here’s what the code is doing step by step:

### 1. **Define People and Houses**
   - The code defines two lists:
     - `people`: A list of individuals (professors): `["Gilderoy", "Pomona", "Minerva", "Horace"]`.
     - `houses`: A list of house names: `["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]`.
   
### 2. **Create Symbolic Representation for Each Person-House Pair**
   - A list of `Symbol` objects is created to represent whether each person belongs to a specific house. For example, `Symbol("GilderoyGryffindor")` represents the proposition that Gilderoy belongs to Gryffindor.

### 3. **Set Up the Knowledge Base**
   - An instance of the `And` class is used to create the `knowledge` base, which will store all the constraints.

### 4. **Add Constraints to the Knowledge Base**
   - **Each Person Belongs to Exactly One House:**
     - For each person, an `Or` statement is added to the knowledge base, representing that the person must belong to one of the four houses (Gryffindor, Hufflepuff, Ravenclaw, or Slytherin).
   
   - **No Person Can Belong to More Than One House:**
     - For each person, additional constraints are added to ensure that if they belong to one house, they cannot belong to another. This is achieved using `Implication` and `Not` statements, e.g., if Gilderoy is in Gryffindor, he cannot be in Hufflepuff, Ravenclaw, or Slytherin.
   
   - **Only One Person Can Belong to Each House:**
     - For each house, constraints are added to ensure that if one person is in that house, no other person can be in the same house.

### 5. **Output the Knowledge Base Formula**
   - `knowledge.formula()` prints the entire logical expression that has been built up, representing all the constraints.

### 6. **Add Specific Conditions to the Knowledge Base**
   - The code adds some additional specific conditions to the knowledge:
     - **Gilderoy belongs to Gryffindor or Ravenclaw**: An `Or` statement is used to represent this constraint.
     - **Pomona does not belong to Slytherin**: This is represented using a `Not` statement.
     - **Minerva belongs to Gryffindor**: This is represented by a `Symbol` statement.

### 7. **Check the Knowledge Base for Entailment**
   - The code iterates through all the symbols and uses the `model_check` function to determine if the knowledge base entails any of the symbols.
   - If a symbol is entailed by the knowledge base, it means that it must be true in every model where the knowledge base holds. The entailed symbols are printed.

### **Purpose of the Code**
The code essentially sets up a logical problem of assigning people to houses with the following rules:
1. Each person must belong to exactly one house.
2. No two people can be assigned to the same house.
3. There are additional specific conditions for some individuals.

The code then tries to determine which assignments are logically deducible based on these constraints. This is an example of using propositional logic to solve constraint satisfaction problems (like scheduling or puzzle solving).

In [ ]:
people = ["Gilderoy", "Pomona", "Minerva", "Horace"]
houses = ["Gryffindor", "Hufflepuff", "Ravenclaw", "Slytherin"]

symbols = []

knowledge = And()

for person in people:
    for house in houses:
        symbols.append(Symbol(f"{person}{house}"))

# Each person belongs to a house.
for person in people:
    knowledge.add(Or(
        Symbol(f"{person}Gryffindor"),
        Symbol(f"{person}Hufflepuff"),
        Symbol(f"{person}Ravenclaw"),
        Symbol(f"{person}Slytherin")
    ))

# Only one house per person.
for person in people:
    for h1 in houses:
        for h2 in houses:
            if h1 != h2:
                knowledge.add(
                    Implication(Symbol(f"{person}{h1}"), Not(Symbol(f"{person}{h2}")))
                )

# Only one person per house.
for house in houses:
    for p1 in people:
        for p2 in people:
            if p1 != p2:
                knowledge.add(
                    Implication(Symbol(f"{p1}{house}"), Not(Symbol(f"{p2}{house}")))
                )

print(knowledge.formula())

knowledge.add(
    Or(Symbol("GilderoyGryffindor"), Symbol("GilderoyRavenclaw"))
)

knowledge.add(
    Not(Symbol("PomonaSlytherin"))
)

knowledge.add(
    Symbol("MinervaGryffindor")
)

for symbol in symbols:
    if model_check(knowledge, symbol):
          print(symbol)

(GilderoyGryffindor ∨ GilderoyHufflepuff ∨ GilderoyRavenclaw ∨ GilderoySlytherin) ∧ (PomonaGryffindor ∨ PomonaHufflepuff ∨ PomonaRavenclaw ∨ PomonaSlytherin) ∧ (MinervaGryffindor ∨ MinervaHufflepuff ∨ MinervaRavenclaw ∨ MinervaSlytherin) ∧ (HoraceGryffindor ∨ HoraceHufflepuff ∨ HoraceRavenclaw ∨ HoraceSlytherin) ∧ (GilderoyGryffindor => (¬GilderoyHufflepuff)) ∧ (GilderoyGryffindor => (¬GilderoyRavenclaw)) ∧ (GilderoyGryffindor => (¬GilderoySlytherin)) ∧ (GilderoyHufflepuff => (¬GilderoyGryffindor)) ∧ (GilderoyHufflepuff => (¬GilderoyRavenclaw)) ∧ (GilderoyHufflepuff => (¬GilderoySlytherin)) ∧ (GilderoyRavenclaw => (¬GilderoyGryffindor)) ∧ (GilderoyRavenclaw => (¬GilderoyHufflepuff)) ∧ (GilderoyRavenclaw => (¬GilderoySlytherin)) ∧ (GilderoySlytherin => (¬GilderoyGryffindor)) ∧ (GilderoySlytherin => (¬GilderoyHufflepuff)) ∧ (GilderoySlytherin => (¬GilderoyRavenclaw)) ∧ (PomonaGryffindor => (¬PomonaHufflepuff)) ∧ (PomonaGryffindor => (¬PomonaRavenclaw)) ∧ (PomonaGryffindor => (¬PomonaSlyt

# *Clue*

The code simulates a deduction problem inspired by the game *Clue* (also known as *Cluedo*), where the objective is to determine the correct person, room, and weapon associated with a crime. The logic-based approach models the problem using propositional logic, defining various symbols and knowledge constraints, and then checks what can be logically deduced from the given information.

### 1. **Defining Symbols**
   - **Characters**: The suspects are represented by the symbols `mustard` (Col. Mustard), `plum` (Prof. Plum), and `scarlet` (Ms. Scarlet).
   - **Rooms**: The possible locations are represented by the symbols `ballroom`, `kitchen`, and `library`.
   - **Weapons**: The potential weapons used in the crime are represented by the symbols `knife`, `revolver`, and `wrench`.
   - **Symbols List**: Combines all characters, rooms, and weapons into a single list (`symbols`) for later use in checking deductions.

### 2. **Defining the `check_knowledge` Function**
   - This function checks which symbols are entailed by the `knowledge` base (representing what is known).
   - It uses the `model_check` function to evaluate whether each symbol can be deduced:
     - If the symbol is entailed by the knowledge (must be true), it prints the symbol with "YES" in green.
     - If the negation of the symbol cannot be confirmed (it could be true or false), it prints the symbol with "MAYBE".

### 3. **Setting Up the Knowledge Base**
   - An instance of `And` is used to create the `knowledge` base, where logical constraints and information are added.
   - **Base Constraints**:
     - At least one of the characters must be involved in the crime: `Or(mustard, plum, scarlet)`.
     - The crime occurred in at least one of the rooms: `Or(ballroom, kitchen, library)`.
     - At least one of the weapons was used: `Or(knife, revolver, wrench)`.

### 4. **Adding Additional Information (Constraints)**
   - **Initial Cards Known to Be False**:
     - `Not(mustard)`, `Not(kitchen)`, and `Not(revolver)` are added, indicating that Col. Mustard is not involved, the kitchen is not the location, and the revolver is not the weapon.
   - **Possible Combinations**:
     - Adds an `Or` condition indicating that at least one of `scarlet`, `library`, or `wrench` is not part of the solution.
   - **Known False Cards**:
     - `Not(plum)` and `Not(ballroom)` indicate that Prof. Plum is not the suspect and the ballroom is not the location.

### 5. **Checking the Knowledge**
   - The `check_knowledge` function is called with the knowledge base, printing whether each character, room, or weapon is a "YES" (definitely true) or "MAYBE" (uncertain based on the current knowledge).

### **Purpose of the Code**
The code performs logical deduction to determine which combinations of person, room, and weapon can be ruled out or confirmed based on the given constraints. It systematically evaluates the possibilities using propositional logic, simulating the process of solving the *Clue* mystery by eliminating suspects, locations, and weapons. This approach can be extended to more complex scenarios by adding more constraints or possibilities.

In [ ]:
import termcolor

mustard = Symbol("ColMustard")
plum = Symbol("ProfPlum")
scarlet = Symbol("MsScarlet")
characters = [mustard, plum, scarlet]

ballroom = Symbol("ballroom")
kitchen = Symbol("kitchen")
library = Symbol("library")
rooms = [ballroom, kitchen, library]

knife = Symbol("knife")
revolver = Symbol("revolver")
wrench = Symbol("wrench")
weapons = [knife, revolver, wrench]

symbols = characters + rooms + weapons


def check_knowledge(knowledge):
    for symbol in symbols:
        if model_check(knowledge, symbol):
            termcolor.cprint(f"{symbol}: YES", "green")
        elif not model_check(knowledge, Not(symbol)):
            print(f"{symbol}: MAYBE")


# There must be a person, room, and weapon.
knowledge = And(
    Or(mustard, plum, scarlet),
    Or(ballroom, kitchen, library),
    Or(knife, revolver, wrench)
)

# Initial cards
knowledge.add(And(
    Not(mustard), Not(kitchen), Not(revolver)
))

# Unknown card
knowledge.add(Or(
    Not(scarlet), Not(library), Not(wrench)
))

# Known cards
knowledge.add(Not(plum))
knowledge.add(Not(ballroom))




In [ ]:
check_knowledge(knowledge)

MsScarlet: YES
library: YES
knife: YES
